In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path  

In [2]:
dem_candidates = pd.read_csv('../datasets/dem_candidates.csv')

In [3]:
#dem_candidates.isna().sum()
#endorsement_cols = dem_candidates.columns[pd.Series(dem_candidates.columns).str.contains('')]
#dem_candidates[endorsement_cols] = dem_candidates[endorsement_cols].fillna(0)

binarydata = dem_candidates.columns[pd.Series(dem_candidates.columns).isin(['Veteran?', 'Self-Funder?', 'LGBTQ?', 'STEM?', 'Obama Alum?', 'Race'])]
dem_candidates[binarydata] = dem_candidates[binarydata].fillna(0)
dem_candidates[binarydata] = dem_candidates[binarydata].replace({'No': 0, 'Yes': 1})
#dem_candidates[endorsement_cols] = dem_candidates[endorsement_cols].replace({'No': 0, 'Yes': 1})
dem_candidates['Race'] = dem_candidates['Race'].replace({'Nonwhite': 1, 'White': 0}) #do not fill NA for now


In [4]:
# endorsement_cols = dem_candidates.columns[pd.Series(dem_candidates.columns).str.contains('Endorsed|Party Support|Support|\\?')]
# dem_candidates[endorsement_cols] = dem_candidates[endorsement_cols].fillna(0)
# dem_candidates[endorsement_cols] = dem_candidates[endorsement_cols].replace({'No': 0, 'Yes': 1})
# dem_candidates['General Status'] = dem_candidates['General Status'].replace({'None': 0, 'On the Ballot': 1})

dem_candidates = dem_candidates[dem_candidates['Race Type'] == 'Regular']

dem_candidates.head(3)

,Candidate,State,District,Office Type,Race Type,Race Primary Election Date,Primary Status,Primary Runoff Status,General Status,Partisan Lean,...,Biden Endorsed?,Warren Endorsed?,Sanders Endorsed?,Our Revolution Endorsed?,Justice Dems Endorsed?,PCCC Endorsed?,Indivisible Endorsed?,WFP Endorsed?,VoteVets Endorsed?,No Labels Support?
0,Anthony White (Alabama),AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,None,None,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Christopher Countryman,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,None,None,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Doug ""New Blue"" Smith",AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,None,None,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#dem_candidates[dem_candidates.duplicated()]
dem_candidates.columns

Index(['Candidate', 'State', 'District', 'Office Type', 'Race Type',
       'Race Primary Election Date', 'Primary Status', 'Primary Runoff Status',
       'General Status', 'Partisan Lean', 'Primary %', 'Won Primary', 'Race',
       'Veteran?', 'LGBTQ?', 'Elected Official?', 'Self-Funder?', 'STEM?',
       'Obama Alum?', 'Party Support?', 'Emily Endorsed?',
       'Guns Sense Candidate?', 'Biden Endorsed?', 'Warren Endorsed? ',
       'Sanders Endorsed?', 'Our Revolution Endorsed?',
       'Justice Dems Endorsed?', 'PCCC Endorsed?', 'Indivisible Endorsed?',
       'WFP Endorsed?', 'VoteVets Endorsed?', 'No Labels Support?'],
      dtype='object')

In [6]:
dem_candidates['LGBTQ?'].value_counts(dropna=False)

0    763
1     36
Name: LGBTQ?, dtype: int64

In [7]:
dem_candidates['Race'].value_counts()

0    600
1    199
Name: Race, dtype: int64

In [8]:
def count_candidates_by_district(df, district_column, new_column_name):
    """
    Adds a column to the DataFrame with the count of candidates in each district.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the candidates and districts.
    district_column (str): The name of the column with district information.
    new_column_name (str): The name of the new column to hold the counts.

    Returns:
    pd.DataFrame: The original DataFrame with an additional column for candidate counts per district.
    """
    # Calculate the number of candidates in each district
    district_counts = df.groupby(district_column).size().reset_index(name=new_column_name)
    
    # Merge the counts back into the original DataFrame
    df_merged = df.merge(district_counts, on=district_column)

    return df_merged


dem_candidates = count_candidates_by_district(dem_candidates, 'District', 'total_runners') #run the above func to get count of ppl in each race and add that as a new col
dem_candidates = dem_candidates[dem_candidates['total_runners']>1] #filter out all the rows where only 1 person ran. 

In [8]:
house = dem_candidates[dem_candidates['Office Type'] == 'Representative'] #just house

# house['dist_num'] = house['District'].str.extract('(\d+)$')
# house['District Abbrev'] = house['State']+ '-' +house['dist_num']
# house

In [ ]:
brookings = pd.read_csv('brookings.csv')
brookings = brookings[(brookings['Candidate.Party'].str.contains('Democrat')) 
    & (brookings['Incumbent'].isnull())
    & (brookings['Primary.Outcome'].isin(['Winner','Loser']))]
brookings['Candidate'] = brookings['Candidate.First.Name'] + ' ' + brookings['Candidate.Last.Name']
brookings

brookings['Candidate'] = brookings['Candidate.First.Name'] + ' ' + brookings['Candidate.Last.Name']
brookings['District'] = brookings['Candidate.State'] + '-' + brookings['Candidate.District'].astype(str)
brookings
#brookings[brookings.duplicated('Candidate', keep=False)]

,Unnamed: 0,Candidate.First.Name,Candidate.Last.Name,Candidate.State,Candidate.District,Candidate.Party,Incumbent,Freshman.Member,Candidate.Website.URL,Candidate.Gender,...,Obama.Mention,Sanders.Mention,Clinton.Mention,Special.Counsel.Mention,Travel.Ban.Mention,SinglePayer,Primary.Outcome,Primary.Runoff.Outcome,Candidate,District
0,1,CAROL,HAFNER,AK,1,Democrat,NaN,NaN,WWW.CAROLHAFNER.COM,Female,...,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,"Yes, candidate supports universal healthcare r...",Loser,NaN,CAROL HAFNER,AK-1
1,2,CHRISTOPHER,CUMINGS,AK,1,Democrat,NaN,NaN,https://ccc4congress.com/,Male,...,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,"Yes, candidate supports universal healthcare r...",Loser,NaN,CHRISTOPHER CUMINGS,AK-1
2,3,DIMITRI,SHEIN,AK,1,Democrat,NaN,NaN,WWW.DIMITRIFORCONGRESS.COM,Male,...,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,"Yes, candidate supports universal healthcare r...",Loser,NaN,DIMITRI SHEIN,AK-1
3,4,ALYSE,GALVIN,AK,1,Democrat,NaN,NaN,WWW.ALYSE4ALASKA.COM,Female,...,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Winner,NaN,ALYSE GALVIN,AK-1
7,8,LIZETTA,MCCONNELL,AL,1,Democrat,NaN,NaN,https://lizzettaforcongress.com/,Female,...,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Loser,NaN,LIZETTA MCCONNELL,AL-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2259,2260,PAUL,DAVIS,WV,3,Democrat,NaN,NaN,http://pauledavisforcongress.com/,Male,...,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Loser,NaN,PAUL DAVIS,WV-3
2260,2261,RICHARD,OJEDA,WV,3,Democrat,NaN,NaN,http://voteojeda.com/,Male,...,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Winner,NaN,RICHARD OJEDA,WV-3
2268,2269,GARY,TRAUNER,WY,0,Democrat,NaN,NaN,https://traunerforwy.com/,Male,...,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Winner,NaN,GARY TRAUNER,WY-0
2275,2276,TRAVIS,HELM,WY,1,Democrat,NaN,NaN,http://www.helmforwyoming.us/,Male,...,NO MENTION,NEUTRAL MENTION,NO MENTION,NO MENTION,NO MENTION,"Yes, candidate supports universal healthcare r...",Loser,NaN,TRAVIS HELM,WY-1


In [11]:
# house['dist_num'] = house['District'].str.extract('(\d+)$')
# house['District Abbrev'] = house['State']+ '-' +house['dist_num']
# house

In [12]:
# brookings = pd.read_csv('candidatesdatafinal2018.csv')
# brookings['Candidate'] = brookings['Candidate.First.Name'] + ' ' + brookings['Candidate.Last.Name']
# brookings['District'] = brookings['Candidate.State'] + '-' + brookings['Candidate.District'].astype(str)
# brookings

In [32]:
# merged_df = pd.merge(brookings, house, left_on=['Candidate', 'District'], right_on=['Candidate', 'District Abbrev'], how='inner')
# merged_df
#merged_df.drop(columns =['Unnamed: 0_x', 'Candidate.First.Name', 'Candidate.Last.Name','Candidate.State', 'Candidate.District', 'Candidate.Gender','Listed.military.service.', 'Party.Category.1','Incumbency',  'Democrat', 'Republican', 'District_x','Unnamed: 0_y', 'State', 'District_y','dist_num'])

In [33]:
# # Assuming you have a list of endorsement column names
# endorsement_columns = endorsement_cols

# def endorsement_effect(data):
#     # Calculating the 'Partisan Lean' percentiles and bins
#     percentiles = data['Partisan Lean'].quantile([0.25, 0.5, 0.75]).tolist()
#     data['Partisan Lean Percentile Bins'] = pd.cut(data['Partisan Lean'], bins=[-np.inf] + percentiles + [np.inf], labels=['0-25%', '26-50%', '51-75%', '76-100%'])

#     # Looping through each endorsement column
#     for endorsement in endorsement_columns:
#         plt.figure(figsize=(12, 8))
#         for i, bin in enumerate(data['Partisan Lean Percentile Bins'].cat.categories, start=1):
#             bin_data = data[data['Partisan Lean Percentile Bins'] == bin]

#             # Calculating success rate for each endorsement status within the bin
#             success_rate_endorsed = bin_data[bin_data[endorsement] == 1]['General Status'].mean()
#             success_rate_not_endorsed = bin_data[bin_data[endorsement] == 0]['General Status'].mean()

#             plt.subplot(2, 2, i)
#             sns.barplot(x=[f'{endorsement[:-1]}', f'Not {endorsement[:-1]}'], y=[success_rate_endorsed, success_rate_not_endorsed])
#             plt.title(f'Success Rate in Partisan Lean Percentile Bin: {bin}')
#             plt.ylabel('Success Rate')

#         plt.suptitle(f'Success Rate Analysis for {endorsement}')
#         plt.tight_layout()
#         plt.show()

In [17]:
# endorsement_effect(house)